In [ ]:
from pathlib import Path

import polars as pl

from utils import Manager


In [14]:
manager = Manager()
run_id_baseline = "q7pgvrou"
run_id_compare = "kv601pjm"
df = manager.preprocess_test_data_df(Path("./wandb_data"))
df.filter(
    (pl.col("run_id") == run_id_compare) & (pl.col("kind") == "pred_mel_speech_ssl")
).select(
    [
        "speaker",
        "filename",
        "wer_kanjikana",
        "utt_gt_kanjikana",
        "utt_recog_kanjikana",
        "cer_kana",
        "utt_gt_kana",
        "utt_recog_kana",
        "per_phoneme",
        "utt_gt_phoneme",
        "utt_recog_phoneme",
        "spk_sim",
    ]
).join(
    df.filter(
        (pl.col("run_id") == run_id_baseline)
        & (pl.col("kind") == "pred_mel_speech_ssl")
    )
    .select(
        [
            "speaker",
            "filename",
            "wer_kanjikana",
            "utt_recog_kanjikana",
            "cer_kana",
            "utt_recog_kana",
            "per_phoneme",
            "utt_recog_phoneme",
            "spk_sim",
        ]
    )
    .rename(
        {
            "wer_kanjikana": "wer_kanjikana_baseline",
            "utt_recog_kanjikana": "utt_recog_kanjikana_baseline",
            "cer_kana": "cer_kana_baseline",
            "utt_recog_kana": "utt_recog_kana_baseline",
            "per_phoneme": "per_phoneme_baseline",
            "utt_recog_phoneme": "utt_recog_phoneme_baseline",
            "spk_sim": "spk_sim_baseline",
        }
    ),
    on=["speaker", "filename"],
    how="left",
).with_columns(
    (pl.col("wer_kanjikana") - pl.col("wer_kanjikana_baseline")).alias(
        "wer_kanjikana_diff"
    ),
    (pl.col("cer_kana") - pl.col("cer_kana_baseline")).alias("cer_kana_diff"),
    (pl.col("per_phoneme") - pl.col("per_phoneme_baseline")).alias("per_phoneme_diff"),
    (pl.col("spk_sim") - pl.col("spk_sim_baseline")).alias("spk_sim_diff"),
).select(
    [
        "speaker",
        "filename",
        "wer_kanjikana_baseline",
        "wer_kanjikana",
        "wer_kanjikana_diff",
        "utt_gt_kanjikana",
        "utt_recog_kanjikana_baseline",
        "utt_recog_kanjikana",
        "cer_kana_baseline",
        "cer_kana",
        "cer_kana_diff",
        "utt_gt_kana",
        "utt_recog_kana_baseline",
        "utt_recog_kana",
        "per_phoneme_baseline",
        "per_phoneme",
        "per_phoneme_diff",
        "utt_gt_phoneme",
        "utt_recog_phoneme_baseline",
        "utt_recog_phoneme",
        "spk_sim_baseline",
        "spk_sim",
        "spk_sim_diff",
    ],
).sort(["speaker", "filename"])

speaker,filename,wer_kanjikana_baseline,wer_kanjikana,wer_kanjikana_diff,utt_gt_kanjikana,utt_recog_kanjikana_baseline,utt_recog_kanjikana,cer_kana_baseline,cer_kana,cer_kana_diff,utt_gt_kana,utt_recog_kana_baseline,utt_recog_kana,per_phoneme_baseline,per_phoneme,per_phoneme_diff,utt_gt_phoneme,utt_recog_phoneme_baseline,utt_recog_phoneme,spk_sim_baseline,spk_sim,spk_sim_diff
str,str,f64,f64,f64,str,str,str,f64,f64,f64,str,str,str,f64,f64,f64,str,str,str,f64,f64,f64
"""F01_kablab""","""ATR503_j01""",50.0,80.0,30.0,"""小さな うなぎ屋 に 熱気 の よう な もの が みなぎる""","""小さな フラギラ に 元気 を 要 だ もの が みなぎる""","""小さな 船 切ら に 元気 を 要 だ もの が 見 できる""",30.434783,47.826087,17.391304,"""チ ー サ ナ ウ ナ ギ ヤ ニ ネ ッ キ ノ ヨ ー …","""チ ー サ ナ フ ラ ギ ラ ニ ゲ ン キ ヲ ヨ ー …","""チ ー サ ナ フ ネ キ リ ラ ニ ゲ ン キ ヲ ヨ …",16.666667,30.952381,14.285714,"""ch i i s a n a u n a g i y a n…","""ch i i s a n a f u r a g i r a…","""ch i i s a n a f u n e k i r i…",0.865022,0.877745,0.012723
"""F01_kablab""","""ATR503_j02""",8.333333,25.0,16.666667,"""泥棒 で も 入っ た か と 一瞬 僕 は 思っ た""","""泥棒 で も 入っ た が と 一瞬 僕 は 思っ た""","""泥棒 で は 入っ た が と 一瞬 ボグ は 思っ た""",4.166667,12.5,8.333333,"""ド ロ ボ ー デ モ ハ イ ッ タ カ ト イ ッ シ …","""ド ロ ボ ー デ モ ハ イ ッ タ ガ ト イ ッ シ …","""ド ロ ボ ー デ ワ ハ イ ッ タ ガ ト イ ッ シ …",2.631579,10.526316,7.894737,"""d o r o b o o d e m o h a i cl…","""d o r o b o o d e m o h a i cl…","""d o r o b o o d e w a h a i cl…",0.852487,0.859571,0.007085
"""F01_kablab""","""ATR503_j03""",40.0,26.666667,-13.333333,"""学生 は レポート を 置く と ちょっと 頭 を 下げ て…","""覚醒 は レポート を オフ と ジョン が 頭 を 下げ …","""覚醒 は レポート を オフ と ちょうど 頭 を 下げ て…",20.689655,13.793103,-6.896552,"""ガ ク セ ー ワ レ ポ ー ト ヲ オ ク ト チ ョ …","""カ ク セ ー ワ レ ポ ー ト ヲ オ フ ト ジ ョ …","""カ ク セ ー ワ レ ポ ー ト ヲ オ フ ト チ ョ …",17.021277,8.510638,-8.510638,"""g a k U s e e w a r e p o o t …","""k a k U s e e w a r e p o o t …","""k a k U s e e w a r e p o o t …",0.896225,0.881552,-0.014673
"""F01_kablab""","""ATR503_j04""",36.363636,27.272727,-9.090909,"""切符 を 買う の は 自動 販売 機 から で ある""","""一方 カウド は 自動 販売 機 から で ある""","""切符 を 買う の は 市場 販売 に カレー で ある""",19.047619,23.809524,4.761905,"""キ ッ プ ヲ カ ウ ノ ワ ジ ド ー ハ ン バ イ …","""イ ッ ポ ー カ ウ ド ワ ジ ド ー ハ ン バ イ …","""キ ッ プ ヲ カ ウ ノ ワ シ ジ ョ ー ハ ン バ …",8.571429,17.142857,8.571429,"""k i cl p u o k a u n o w a j i…","""i cl p o o k a u d o w a j i d…","""k i cl p u o k a u n o w a sh …",0.829314,0.805874,-0.02344
"""F01_kablab""","""ATR503_j05""",0.0,7.692308,7.692308,"""都会 で は 出会う 人 の ほとんど が 見知ら ぬ 人 …","""都会 で は 出会う 人 の ほとんど が 見知ら ぬ 人 …","""魚介 で は 出会う 人 の ほとんど が 見知ら ぬ 人 …",0.0,8.0,8.0,"""ト カ イ デ ワ デ ア ウ ヒ ト ノ ホ ト ン ド …","""ト カ イ デ ワ デ ア ウ ヒ ト ノ ホ ト ン ド …","""ギ ョ カ イ デ ワ デ ア ウ ヒ ト ノ ホ ト ン …",0.0,2.222222,2.222222,"""t o k a i d e w a d e a u h I …","""t o k a i d e w a d e a u h I …","""gy o k a i d e w a d e a u h I…",0.844487,0.837288,-0.007199
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""M04_kablab""","""ATR503_j49""",28.571429,57.142857,28.571429,"""雨 の ため か 野鳥 が 群がっ て 奇声 を あげ て …","""雨 の ため が 野生 が 群がっ て 気性 を 上げ て …","""雨 の ため が 野生 株 だ が って 帰省 を 開け て…",20.0,28.0,8.0,"""ア メ ノ タ メ カ ヤ チ ョ ー ガ ム ラ ガ ッ …","""ア メ ノ タ メ ガ ヤ セ ー ガ ム ラ ガ ッ テ …","""ア メ ノ タ メ ガ ヤ セ ー カ ブ ダ ガ ッ テ …",17.073171,19.512195,2.439024,"""a m e n o t a m e k a y a ch o…","""a m e n o t a m e g a y a s e …","""a m e n o t a m e g a y a s e …",0.891449,0.885688,-0.005761
"""M04_kablab""","""ATR503_j50""",84.615385,69.230769,-15.384615,"""部屋 いっぱい に タバコ の 濃霧 が 立ち込め ゆるやか…","""部屋 一杯 に 卵 の 方 も が 出し 込め 夕焼け が …","""部屋 いっぱい に タマコドゥオウム が 出し 込め 夕焼 …",46.666667,43.333333,-3.333333,"""ヘ ヤ イ ッ パ イ ニ タ バ コ ノ ノ ー ム ガ …","""ヘ ヤ イ ッ パ イ ニ タ マ ゴ ノ ホ ー モ ガ …","""ヘ ヤ イ ッ パ イ ニ タ マ コ ド ゥ オ ー ム …",28.301887,26.415094,-1.886792,"""h e y a i cl p a i n i t a b a…","""h e y a i cl p a i n i t a m a…","""h e y a i cl p a i n i t a m a…",0.835275,0.843113,0.007838
"""M04_kablab""","""ATR503_j51""",166.666667,133.333333,-33.333333,"""畑 は 干上がり 土 は 割れる""","""あ たて は ひや が に 口 を あわ り る""","""あ だけ は 冷や がり 土 と あわ り る""",57.142857,42.857143,-14.285714,"""ハ タ ケ ワ ヒ ア ガ リ ツ チ ワ ワ レ ル""","""ア タ テ ワ ヒ ヤ ガ ニ ク チ ヲ ア ワ リ ル""",""